In [25]:
from collections import Counter
import pandas as pd
import numpy as np
import re
from pprint import pprint
import db_mysql as db_mysql

In [28]:
import MySQLdb

hostname = "localhost"
username = "root"
password = ""
db_name  = "drama_ojol"

# Open database connection
db = MySQLdb.connect(hostname,username,password,db_name)
db.set_character_set('utf8')

# prepare a cursor object using cursor() method
cursor = db.cursor()
cursor.execute('SET NAMES utf8;')
cursor.execute('SET CHARACTER SET utf8;')
cursor.execute('SET character_set_connection=utf8;')

def executeSql(sql):
    return cursor.execute(sql);

def fetch(type='all'):
    if type=='all':
        return cursor.fetchall();
    else:
        return cursor.fetchone();
def commit():
    db.commit();

def escapeString(string):
    return MySQLdb.escape_string(string);

# Preprocessing Mentions

In [2]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = re.sub('\.+','.',tweet);
    tweet = re.sub('\,+',',',tweet);
    tweet = re.sub('\"+','"',tweet); #Remove tanda ""
    tweet = tweet.replace("/ ","/");
    tweet = tweet.replace("www. ","www.");
    tweet = re.sub(r'RT', '', tweet);
    tweet = re.sub(r'http\S+', '', tweet, flags=re.MULTILINE);
    tweet = re.sub(r'@([A-Za-z0-9_]+)', '', tweet, flags=re.MULTILINE);
    tweet = re.sub(r'pic.twitter.com/\S+', '', tweet, flags=re.MULTILINE);
    tweet = tweet.replace("?"," ? ");
    tweet = tweet.replace("-"," ");
    tweet = tweet.replace("/"," ");
    tweet = tweet.replace("["," ");
    tweet = tweet.replace("]"," ");
    tweet = tweet.replace("."," ");
    tweet = tweet.replace(","," ");
    tweet = re.sub(r'[^a-z0-9.,?# -]', ' ', tweet, flags = re.IGNORECASE|re.MULTILINE)
    tweet = re.sub(" \d+", " ", tweet)
    tweet = re.sub(r'( +)', ' ', tweet, flags = re.IGNORECASE|re.MULTILINE)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

# Read the tweets one by one and process it

In [6]:
dokumen = []
docs, index = [], []
fp = open('tes.csv', 'r')
line = fp.readline()
while line:
    processedTweet = processTweet(line)
    dokumen.append(processedTweet.split(' '))
    line = fp.readline()
fp.close()

In [7]:
datatest = pd.read_csv("data/kategori/datatest.csv",sep=",", header=None)
datatest.columns = ['kategori']

# print(datatest['kategori'][1])
# if(datasets['kategori'][0]=="aplikas"):
#     print("ada")

# Make dataframe containing list of word each category

In [8]:
df_index = pd.DataFrame({
    'kata':datatest['kategori']
})

# Count word accurance in each document

In [9]:
for i in range(len(dokumen)):
    terdeteksi = []
    for kata in datatest['kategori']:
        if kata in dokumen[i]:
            nilai=1
        else:
            nilai=0
        terdeteksi.append(nilai)
        df_index[i+1] = pd.Series(terdeteksi)

In [10]:
df_index.to_csv("data/matriks_datatest.csv", index=False)
# df_tmp = pd.read_csv("data/matriks_datatest.csv",sep=",")
# df_index = df_tmp.drop(['kata'], axis=1)
df_index

,kata,1,2,3,4,5,6,7,8,9,...,3390,3391,3392,3393,3394,3395,3396,3397,3398,3399
0,tarif,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tarifnya,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,murah,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,mahal,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,harga,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,argo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,voucher,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,diskon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,ongkir,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,ongkos kirim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Open the aggregration matrix

In [11]:
category = pd.read_csv("data/aggregasi_matriks.csv",sep=",", header=0)
list_cat = category.drop(['kategori'], axis=1)

In [12]:
list_category = list_cat.values.tolist()
for category in list_category:
    for item in category:
        float(item)

# Assign matrix of each category into variables

In [13]:
cat_harga = []
cat_aplikasi = []
cat_driver = []
cat_penumpang = []
cat_perusahaan = []
cat_harga = np.array(list_category[0])
cat_aplikasi = np.array(list_category[1])
cat_driver = np.array(list_category[2])
cat_penumpang = np.array(list_category[3])
cat_perusahaan = np.array(list_category[4])

# Convert datatest matrix into numpy array

In [14]:
temp = df_index.drop(['kata'], axis=1)
temp_datatest = temp.values.tolist()
datatest = np.array(temp_datatest)
# datatest[:,0].shape

# Fungsi Cosine Similarity

In [15]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

# Categorisasi dengan cosine

In [31]:
total_label = []
total_datatest = np.size(datatest,1)
for i in range(0,total_datatest):
    harga = cos_sim(datatest[:,i], cat_harga)
    aplikasi = cos_sim(datatest[:,i], cat_aplikasi)
    driver = cos_sim(datatest[:,i], cat_driver)
    penumpang = cos_sim(datatest[:,i], cat_penumpang)
    perusahaan = cos_sim(datatest[:,i], cat_perusahaan)
    maximal = max(harga,aplikasi,driver,penumpang,perusahaan)
    if maximal == harga:
        label = "harga"
    if maximal == aplikasi:
        label = "aplikasi"
    elif maximal == driver:
        label = "driver"
    elif maximal == penumpang:
        label = "penumpang"
    elif maximal == perusahaan:
        label = "perusahaan"
    
    sql = '''UPDATE opinions
    		 SET cat_result="%s" WHERE id=%d''' % \
    		 (label,i+1);
    executeSql(sql);
    commit();

C:\Users\Brandon\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
